In [127]:
import pandas as pd

from pathlib import Path

In [88]:
def dataframe_from_csv(target):
      return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
  return pd.concat([dataframe_from_csv(x) for x in targets])

In [89]:
data_path = 'dataset'

train_files = sorted([x for x in Path(f'{data_path}/train/').glob('*.csv')])
val_files = sorted([x for x in Path(f'{data_path}/val/').glob('*.csv')])

train = dataframe_from_csvs(train_files)
val = dataframe_from_csvs(val_files)
test = pd.read_csv(f'{data_path}/test.csv')
print(f'train: {len(train)}')
print(f'validation: {len(val)}')
print(f'test: {len(test)}')

train: 62564
validation: 7820
test: 7820


In [90]:
train['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
val['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
test['leaktype']=""

In [93]:
def normalize(df):
    """
    column 별로 정규화를 시키는 함수입니다.
    이때, 정규화 방식은 표준편차가 아닌 최대 - 최소로 구하였습니다.
    진행한 이유는 특정 column 값이 큰 영향을 끼칠 수 없도록 제한하기 위함입니다.

    Args:
        df (DataFrame): csv 파일을 pandas 로 읽은 데이터

    Returns:
        result (DataFrame) : df 를 column 별로 정규화한 데이터
    """
    result = df.copy()
    
    for feature_name in df.columns:
        # site, sid, leaktype 은 정규화 대상이 아니므로, 값을 그대로 저장하고 넘어갑니다.
        if feature_name in ['site', 'sid', 'leaktype']:
            result[feature_name] = df[feature_name]
            continue
    
        # C01 ~ C26
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value) * 100
        
    return result

In [108]:
def num2alpha(char_dict):
    """
    숫자를 자연어(영어 형태)로 변환하는 함수입니다.
    
    영어를 택한 이유는 크게 2가지입니다.
    - C01 ~ C26 이 26개로 알파벳 개수와 동일하다.
    - 현재 NLP 에서 가장 뛰어난 모델은 대부분 영어 데이터에 최적화되어 있다.
    
    결론부터 말씀드리면 결과는 다음 예시처럼 나옵니다.
    
    예시: 'site: S-4784025026. sid: S-0359369085186035. aa.... zz.'
    
    변환하는 방식은 다음과 같습니다.
    
    1. 각 row 별로 백분율을 구합니다.
    2. 그리고 백분율에 400 을 곱합니다.
    여기서 400 을 곱하는 이유는 'BERT'의 최대 입력 길이를 고려했기 때문입니다.
    최대 길이가 512 인데, site 와 sid 를 제일 앞에 표기한 길이를 대강 112 로 정했습니다.
    그리고 그 후, 나머지 길이 400 을 나머지 row 끼리 나눠서 알파벳을 나열하는 구조입니다.
    3. 이때, 알파벳별로 간격을 두었습니다.

    Args:
        char_dict (Dict): C01 ~ C26 값을 모두 저장한 딕셔너리 자료형

    Returns:
        str : 자연어로 변환된 문자열
    """
    sentence = list()
    
    s = sum(char_dict.values())
    for k, v in char_dict.items():
        a = round(v / s * 400) * chr(96 + int(k[1:]))
        sentence.append(a)

    return ' '.join(sentence)
    

In [117]:
def tablet2nlp(df):
    """
    : tablet 데이터를 자연어 데이터로 변환하는 함수
    
    Args:
        df (DataFrame): csv 파일을 읽은 데이터

    Returns:
        DataFrame: 자연어로 변환된 데이터
    """
    nlp_list = list()
    
    for i, v in df.iterrows():
        nlp_data = list() # 최종적으로 자연어를 저장하는 변수
        char_dict = dict()  # 'C01' ~ 'C26' 을 저장하는 변수

        for c in df.columns:
            if c.startswith('C'):
                char_dict[c] = v[c]
                
            elif c.startswith('si'): # site, sid
                nlp_data.append(f'{c}: {v[c]}.')
        
        nlp_data.append(num2alpha(char_dict))
        nlp_list.append(' '.join(nlp_data) + '.')
    
    df['nlp'] = nlp_list

    # label 과 자연어 데이터 이외에는 모두 삭제합니다.
    df = df[['leaktype', 'nlp']]

    return df                

In [124]:
def data2nlp(df):
    """
    실질적으로 부르는 함수.
    1. 정규화하고,
    2. 자연어로 변환한다.
    
    간단히 표현하고자 있는 함수입니다.
    """
    df = normalize(df)
    return tablet2nlp(df)

In [126]:
# 데이터를 모두 자연어 형태로 변환한다.
nlp_train = data2nlp(train)
nlp_val = data2nlp(val)
nlp_test = data2nlp(test)

In [128]:
# 데이터를 모두 저정해둡니다. 이때, index=False
nlp_train.to_csv('dataset/nlp/train.csv', index=False)
nlp_val.to_csv('dataset/nlp/val.csv', index=False)
nlp_test.to_csv('dataset/nlp/test.csv', index=False)